In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd


"""
Checks for an existing data frame, and if one does not exist, creates it
"""
try:
    df = pd.read_csv('./records.csv')
except Exception as e:
    data = {
        "Date" : [],
        "Description" : [],
        "Amount" : [],
        "Type" : [] 
    }

    df = pd.DataFrame(data)

class BudgetTrackerApp:
    def __init___(self, root):
        self.root = root
        self.root.title("Budget Tracker")


In [ ]:
root = tk.Tk()
app = BudgetTrackerApp(root)
root.mainloop()